In [1]:
import pandas as pd
from bokeh.io import curdoc
from bokeh.plotting import figure, show
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.models import CategoricalColorMapper
from bokeh.palettes import Spectral6
from bokeh.layouts import widgetbox, row, gridplot
from bokeh.models import Slider, Select

In [2]:
data = pd.read_csv("national-history.csv", parse_dates=[0])
data.head()

,date,death,deathIncrease,inIcuCumulative,inIcuCurrently,hospitalizedIncrease,hospitalizedCurrently,hospitalizedCumulative,negative,negativeIncrease,onVentilatorCumulative,onVentilatorCurrently,positive,positiveIncrease,states,totalTestResults,totalTestResultsIncrease
0,2021-03-07,515151.0,842,45475.0,8134.0,726,40199.0,776361.0,74582825.0,131835,4281.0,2802.0,28756489.0,41835,56,363825123,1170059
1,2021-03-06,514309.0,1680,45453.0,8409.0,503,41401.0,775635.0,74450990.0,143835,4280.0,2811.0,28714654.0,60015,56,362655064,1430992
2,2021-03-05,512629.0,2221,45373.0,8634.0,2781,42541.0,775132.0,74307155.0,271917,4275.0,2889.0,28654639.0,68787,56,361224072,1744417
3,2021-03-04,510408.0,1743,45293.0,8970.0,1530,44172.0,772351.0,74035238.0,177957,4267.0,2973.0,28585852.0,65487,56,359479655,1590984
4,2021-03-03,508665.0,2449,45214.0,9359.0,2172,45462.0,770821.0,73857281.0,267001,4260.0,3094.0,28520365.0,66836,56,357888671,1406795


In [3]:
# Make a list of the unique values from the region column: regions_list
death_list = data['date'].dt.strftime('%Y-%m-%d').unique().tolist()

# Make a color mapper: color_mapper
colors = ["red","green","blue","black","orange","brown","grey","purple","yellow","cyan","pink","peru"]
color_mapper = CategoricalColorMapper(factors=death_list[0:12], palette=colors)

In [4]:
data.loc[[0,1,2,3,4]].deathIncrease

0     842
1    1680
2    2221
3    1743
4    2449
Name: deathIncrease, dtype: int64

In [5]:
#Make the ColumnDataSource: source
source = ColumnDataSource(data={
    "x": data.loc[[0,1,2,3,4,5]].date,
    "y": data.loc[[0,1,2,3,4,5]].death,
    "states" : data.loc[[0,1,2,3,4,5]].states
})
source

ColumnDataSource(id='1003', ...)

In [6]:
# Create the figure: plot
plot = figure(title='Covid-19 Death Rate', x_axis_label='Date', y_axis_label='Death',
           plot_height=400, plot_width=700, x_axis_type="datetime", tools=[HoverTool(tooltips='@x')])

# Add a circle glyph to the figure p
# plot.circle(x='x', y='y', source=source, fill_alpha=0.8,
#            color=dict(field='states', transform=color_mapper), legend_label='DeathIncrease')
plot.circle(x='x', y='y', source=source, fill_alpha=0.8, legend_label='DeathIncrease')

# Set the legend and axis attributes
plot.legend.location = 'bottom_left'


# Define the callback function: update_plot
def update(attr, old, new):
    # set the `dt` name to `slider.value` and `source.data = new_data`
    dt = slider.value
    date = date_select.value
    death = death_select.value
    # Label axes of plot
    plot.xaxis.axis_label = c1
    plot.yaxis.axis_label = c2
    # new data
    new_data = {
        'Date'        : data.loc[date,c1].date,
        'Death'       : data.loc[date,c2].death,
        "deathIncrease" : data.loc[date].deathIncrease
    }
    source.data = new_data

# # Make a slider object: slider
# slider = Slider(start=1970, end=2010, step=1, value=1970, title='Year')
# slider.on_change('value',update)

# Make dropdown menu for x and y axis
# Create a dropdown Select widget for the x data: x_select
x_select = Select(
    options=['deathIncrease', 'hospitalizedIncrease', 'negativeIncrease', 'positiveIncrease'],
    value='date',
    title='Date_axis'
)
# Attach the update_plot callback to the 'value' property of x_select
x_select.on_change('value', update)

# Create a dropdown Select widget for the y data: y_select
y_select = Select(
    options=['deathIncrease', 'hospitalizedIncrease', 'negativeIncrease', 'positiveIncrease'],
    value='death',
    title='Death_axis'
)
# Attach the update_plot callback to the 'value' property of y_select
y_select.on_change('value', update)

# Create layout and add to current document
layout = row(widgetbox(x_select, y_select), plot)
curdoc().add_root(layout)

show(layout)

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html

